# Fine Tune LLM for Text Summary
- https://www.kaggle.com/code/mitanshuchakrawarty/fine-tune-llm-for-text-summary

## 1. 환경 셋업

In [1]:
# !pip install -q -U bitsandbytes
# !pip install transformers==4.31
# # !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install peft
# !pip install -q datasets
# !pip install -qqq trl==0.7.1

In [2]:
import torch
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from datasets import load_dataset, load_metric
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import warnings
warnings.filterwarnings("ignore")

/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. 데이터 셋 준비
### 데이터 셋 다운로드

In [3]:
huggingface_dataset_name = "cnn_dailymail"

dataset = load_dataset(huggingface_dataset_name, "3.0.0")
dataset

Generating test split: 100%|██████████| 11490/11490 [00:00<00:00, 75243.14 examples/s]


DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [4]:
sample = dataset["train"][1]
print(f"""Article (excerpt of 500 characters, total length: {len(sample["article"])}):""")
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])

Article (excerpt of 500 characters, total length: 4051):
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

Summary (length: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


### 데이터텟 변형

In [5]:
def format_instruction(dialogue: str, summary: str):
    return f"""### Instruction:
Summarize the following conversation.

### Input:
{dialogue.strip()}

### Summary:
{summary}
""".strip()

def generate_instruction_dataset(data_point):

    return {
        "article": data_point["article"],
        "highlights": data_point["highlights"],
        "text": format_instruction(data_point["article"],data_point["highlights"])
    }

def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset).remove_columns(['id'])
    )

In [6]:
## APPLYING PREPROCESSING ON WHOLE DATASET
dataset["train"] = process_dataset(dataset["train"])
dataset["test"] = process_dataset(dataset["validation"])
dataset["validation"] = process_dataset(dataset["validation"])

# Select 1000 rows from the training split
train_data = dataset['train'].shuffle(seed=42).select([i for i in range(1000)])

# Select 100 rows from the test and validation splits
test_data = dataset['test'].shuffle(seed=42).select([i for i in range(100)])
validation_data = dataset['validation'].shuffle(seed=42).select([i for i in range(100)])

train_data,test_data,validation_data

Map: 100%|██████████| 13368/13368 [00:01<00:00, 9086.61 examples/s]


(Dataset({
     features: ['article', 'highlights', 'text'],
     num_rows: 1000
 }),
 Dataset({
     features: ['article', 'highlights', 'text'],
     num_rows: 100
 }),
 Dataset({
     features: ['article', 'highlights', 'text'],
     num_rows: 100
 }))

## 3. 베이스 모델 준비

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/Llama-3-Open-Ko-8B"
# model_id =  "NousResearch/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards: 100%|██████████| 6/6 [00:05<00:00,  1.05it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 베이스 모델 추론

In [8]:
index = 2

dialogue = test_data['article'][index]
summary = test_data['highlights'][index]

prompt = f"""
Summarize the following conversation.

### Input:
{dialogue}

### Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=100,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

### Input:
A federal appeals court has given new life to a Holocaust survivor's claim that the University of Oklahoma is unjustly harboring a Camille Pissarro painting that the Nazis stole from her father during World War II. The 2nd U.S. Circuit Court of Appeals in Manhattan has directed a lower-court judge to consider whether the lawsuit she threw out should be transferred to Oklahoma, saying she has authority to do so. The court's order on Thursday came as the school found itself amid a racial controversy after video of fraternity students engaged in a racist chant spread across the Internet. Dr. Leone-Noelle Meyer maintained she is entitled to Pissarro's 1886 'Shepherdess Bringing in Sheep' because it belonged to her father when it was taken by the Nazis as Germany moved across France . University President David Boren ordered a f

### 모델을 LoRA 를 사용할 수 있게 준비

In [9]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

### PEFT 모델로 변환

In [10]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], #specific to Llama models.
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)



In [11]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():

        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


print_trainable_parameters(model)

trainable params: 13631488 || all params: 4554231808 || trainable%: 0.29931475986915773


## 4. 훈련 준비

In [12]:
from transformers import TrainingArguments

OUTPUT_DIR = "llama2-docsum-adapter"
epoch = 1

training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=epoch,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    # report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [13]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=validation_data,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
)


Map: 100%|██████████| 100/100 [00:00<00:00, 2069.93 examples/s]


## 5. 훈련 실행

In [14]:
trainer.train()

ValueError: You can't train a model that has been loaded in 8-bit precision on a different device than the one you're training on. Make sure you loaded the model on the correct device using for example `device_map={'':torch.cuda.current_device() or device_map={'':torch.xpu.current_device()}

## 6. 모델 저장

In [22]:
peft_model_path="./peft-dialogue-summary"

trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('./peft-dialogue-summary/tokenizer_config.json',
 './peft-dialogue-summary/special_tokens_map.json',
 './peft-dialogue-summary/tokenizer.json')

In [23]:
! ls {peft_model_path} -al -h

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
total 66M
drwxrwxr-x 2 ec2-user ec2-user  165 Jun 25 12:29 .
drwxrwxr-x 5 ec2-user ec2-user  138 Jun 25 12:29 ..
-rw-rw-r-- 1 ec2-user ec2-user  650 Jun 25 12:30 adapter_config.json
-rw-rw-r-- 1 ec2-user ec2-user  65M Jun 25 12:30 adapter_model.safetensors
-rw-rw-r-- 1 ec2-user ec2-user 5.0K Jun 25 12:30 README.md
-rw-rw-r-- 1 ec2-user ec2-user  434 Jun 25 12:30 special_tokens_map.json
-rw-rw-r-- 1 ec2-user ec2-user  695 Jun 25 12:30 tokenizer_config.json
-rw-rw-r-- 1 ec2-user ec2-user 1.8M Jun 25 12:30 tokenizer.json


## 7.Inference

In [24]:
from transformers import TextStreamer
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer

### 훈련된 모델 로딩

In [25]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

peft_model_dir = "peft-dialogue-summary"

# load base LLM model and tokenizer
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 모델 추론

In [26]:
index = 51

dialogue = train_data['article'][index][:10000]
summary = train_data['highlights'][index]

prompt = f"""
Summarize the following conversation.

### Input:
{dialogue}

### Summary:
"""

def inference_model(tokenizer,prompt, model ):
    input_ids = tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()
    outputs = model.generate(input_ids=input_ids, max_new_tokens=200, )
    output= tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

    dash_line = '-'.join('' for x in range(100))
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'TRAINED MODEL GENERATED TEXT :\n{output}')



In [20]:
inference_model(tokenizer = tokenizer,
                prompt = prompt, 
                model = trained_model)
    

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

### Input:
You might expect polar bears, the Artic Circle's apex predators, to be dab hands at dancing on ice. But as this specimen in Svalbard shows, even after thousands of years of evolutionary adaptation, some still suffer from two-left feet on the frozen ocean. Heinrich Eggenfellner, a 49-year-old videographer from Norway, said: 'I have encountered polar bears many times every year since I live up here and am used to them. 'This episode, however, was extraordinary.' Born slippy: A polar walks across thin sea ice in Svalbard, Norway, where it was caught on camera looking very unsteady on its feet as it made its way across the slippery surface . Spreading its weight: The polar bear does its best not to collapse through the fragile ice by spreading out . In its element: The beast finally gave up and pushed a hole in the ice to dive 

In [21]:
inference_model(tokenizer = tokenizer,
                prompt = prompt, 
                model = model)
    

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

### Input:
You might expect polar bears, the Artic Circle's apex predators, to be dab hands at dancing on ice. But as this specimen in Svalbard shows, even after thousands of years of evolutionary adaptation, some still suffer from two-left feet on the frozen ocean. Heinrich Eggenfellner, a 49-year-old videographer from Norway, said: 'I have encountered polar bears many times every year since I live up here and am used to them. 'This episode, however, was extraordinary.' Born slippy: A polar walks across thin sea ice in Svalbard, Norway, where it was caught on camera looking very unsteady on its feet as it made its way across the slippery surface . Spreading its weight: The polar bear does its best not to collapse through the fragile ice by spreading out . In its element: The beast finally gave up and pushed a hole in the ice to dive 